# Finetuning GPT-3.5 - CW Coherence.ipynb

Sources: https://blog.futuresmart.ai/fine-tuning-gpt-35-a-step-by-step-guide

In [ ]:
import pandas as pd
df = pd.read_csv("bank_support_train.csv")
df.head(5)


### Format Dataset

In [3]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Top Category": "' + row['Top Category'] + '", "Sub Category": "' + row['Sub Category'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['Support Query']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

dataset = pd.read_csv('/content/back_support_train.csv')
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']


[{'role': 'user',
  'content': 'Can you explain the monthly maintenance fee on my account?'},
 {'role': 'assistant',
  'content': '{"Top Category": "Fees and Charges", "Sub Category": "Understanding Fees"}'}]

In [4]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])


{'Top Category': 'Fees and Charges', 'Sub Category': 'Understanding Fees'}

Create Train and Val Set

In [5]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=dataset['Top Category'],
    random_state=42  # for reproducibility
)


In [7]:
type(train_data[0])


dict

Create JSONL file

In [8]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)


In [ ]:
# Get and set API key
with open('C:/Users/ijyli/Documents/OpenAI/anlp23-project.txt', 'r') as file:
    my_api_key = file.read()

from openai import OpenAI
client = OpenAI(api_key=my_api_key)


Upload Training and Validation File

In [10]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)


Training file id: file-lH871kh5kCsKUV31vYjbxe7C
Validation file id: file-jg2GAagSKpGN7mgicEiKdayI


Create Finetuning Job

In [11]:
suffix_name = "yt_tutorial"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response


FineTuningJob(id='ftjob-dyySU8thbv5LBSdq1KrXZiki', created_at=1699779814, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=[], status='validating_files', trained_tokens=None, training_file='file-lH871kh5kCsKUV31vYjbxe7C', validation_file='file-jg2GAagSKpGN7mgicEiKdayI')

### All Finetuning  Jobs

In [16]:
client.fine_tuning.jobs.list(limit=10)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-dyySU8thbv5LBSdq1KrXZiki', created_at=1699779814, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=[], status='running', trained_tokens=None, training_file='file-lH871kh5kCsKUV31vYjbxe7C', validation_file='file-jg2GAagSKpGN7mgicEiKdayI'), FineTuningJob(id='ftjob-h5P9gmmDXysjbOD99jz8TtYv', created_at=1699776576, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc', finished_at=1699777187, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=['file-94bTw4zTuPmEg5cDx8UbIg7S'], status='succeeded', trained_tokens=9123, training_file='file-xWkwYEZgnD9boGLZAR

### Retrieve Specific Job

In [14]:
response = client.fine_tuning.jobs.retrieve("ftjob-h5P9gmmDXysjbOD99jz8TtYv")
response


FineTuningJob(id='ftjob-h5P9gmmDXysjbOD99jz8TtYv', created_at=1699776576, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc', finished_at=1699777187, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=['file-94bTw4zTuPmEg5cDx8UbIg7S'], status='succeeded', trained_tokens=9123, training_file='file-xWkwYEZgnD9boGLZAR6ebpYO', validation_file='file-R4ZUTOca4nq1l3yzEhDIkbXR')

In [15]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)



Fine-tuned model id: ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc


### Test Finetuned Model

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):

    formatted_message = [
        {
            "role": "user",
            "content": row['Support Query']
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content


In [18]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")


In [19]:
test_df = pd.read_csv("test_queries.csv")
store_predictions(test_df, fine_tuned_model_id)


fine_tuned_model_id ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc
